In [31]:
import pandas as pd

# Load the dataset
df = pd.read_csv('merged_data (1).csv')

# Display the first few rows and info
print(df.head())
print(df.info())

# Check unique values in 'ioc' to confirm it's country codes
print(df['ioc'].unique())

   ranking_date  rank  points  player_id name_first name_last hand  ioc  \
0      20240101     1   11245   104925.0      Novak  Djokovic    R  SRB   
1      20240101     2    8855   207989.0     Carlos   Alcaraz    R  ESP   
2      20240101     3    7600   106421.0     Daniil  Medvedev    R  RUS   
3      20240101     4    6490   206173.0     Jannik    Sinner    R  ITA   
4      20240101     5    4805   126094.0     Andrey    Rublev    R  RUS   

   height  
0   188.0  
1   183.0  
2   198.0  
3   191.0  
4   188.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92341 entries, 0 to 92340
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ranking_date  92341 non-null  int64  
 1   rank          92341 non-null  int64  
 2   points        92341 non-null  int64  
 3   player_id     92301 non-null  float64
 4   name_first    92301 non-null  object 
 5   name_last     92301 non-null  object 
 6   hand          92301 

In [32]:
# Check date range
print(df['ranking_date'].min(), df['ranking_date'].max())

# Filter for the most recent ranking date
latest_date = df['ranking_date'].max()
df_latest = df[df['ranking_date'] == latest_date]

# Idea 1 & 2: Total points and Player count by country
country_stats = df_latest.groupby('ioc').agg(
    total_points=('points', 'sum'),
    player_count=('player_id', 'count'),
    avg_height=('height', 'mean'),
    avg_rank=('rank', 'mean')
).reset_index()

# Calculate Top 100 players count
top_100_stats = df_latest[df_latest['rank'] <= 100].groupby('ioc').size().reset_index(name='top_100_count')

# Merge top 100 count
country_stats = country_stats.merge(top_100_stats, on='ioc', how='left').fillna(0)

# Sort by total points to see top countries
print(country_stats.sort_values(by='total_points', ascending=False).head(10))
print(country_stats.describe())

20240101 20241230
     ioc  total_points  player_count  avg_height     avg_rank  top_100_count
49   ITA         30036           182  183.547170  1080.714286            9.0
101  USA         29990           218  182.954545  1117.527523            8.0
34   FRA         25054           164  185.310680   912.439024           12.0
31   ESP         17481            96  182.188679  1046.447917            6.0
4    AUS         15731            79  183.527273  1032.202532            9.0
85   RUS         15396            71  186.736842  1043.816901            5.0
3    ARG         14717            88  180.843750   907.238636            8.0
37   GER         14480            93  187.229167  1058.247312            4.0
35   GBR          8383            82  184.680000  1112.109756            3.0
89   SRB          7130            30  185.833333  1034.266667            3.0
       total_points  player_count  avg_height     avg_rank  top_100_count
count    106.000000    106.000000  106.000000   106.000000   

In [33]:
# Filter for the latest date again to be sure
latest_date = df['ranking_date'].max()
df_latest = df[df['ranking_date'] == latest_date].copy()

# Clean height: replace 0 with NaN
df_latest['height'] = df_latest['height'].replace(0, pd.NA)

# 1. Basic Aggregations
agg_stats = df_latest.groupby('ioc').agg(
    total_points=('points', 'sum'),
    player_count=('player_id', 'count'),
    average_rank=('rank', 'mean'),
    average_height=('height', 'mean')
).reset_index()

# 2. Top 100 Count
top_100 = df_latest[df_latest['rank'] <= 100].groupby('ioc').size().reset_index(name='top_100_players')

# 3. Best Player Info
# Sort by rank (ascending) to get the best player first
df_sorted = df_latest.sort_values('rank')
best_players = df_sorted.groupby('ioc').first()[['name_first', 'name_last', 'rank']].reset_index()
best_players['best_player_name'] = best_players['name_first'] + ' ' + best_players['name_last']
best_players = best_players.rename(columns={'rank': 'best_player_rank'})

# Merge everything
final_df = agg_stats.merge(top_100, on='ioc', how='left')
final_df = final_df.merge(best_players[['ioc', 'best_player_name', 'best_player_rank']], on='ioc', how='left')

# Fill NaNs for Top 100 count with 0
final_df['top_100_players'] = final_df['top_100_players'].fillna(0)

# Round numerical columns for display
final_df['average_height'] = final_df['average_height'].round(1)
final_df['average_rank'] = final_df['average_rank'].round(0)

# Sort by Total Points for preview
final_df = final_df.sort_values(by='total_points', ascending=False)

# Save to CSV
final_df.to_csv('tableau_choropleth_data.csv', index=False)

print(final_df.head())

     ioc  total_points  player_count  average_rank  average_height  \
49   ITA         30036           182        1081.0           183.5   
101  USA         29990           218        1118.0           183.0   
34   FRA         25054           164         912.0           185.3   
31   ESP         17481            96        1046.0           182.2   
4    AUS         15731            79        1032.0           183.5   

     top_100_players best_player_name  best_player_rank  
49               9.0    Jannik Sinner                 1  
101              8.0     Taylor Fritz                 4  
34              12.0      Ugo Humbert                14  
31               6.0   Carlos Alcaraz                 3  
4                9.0   Alex De Minaur                 9  


In [34]:
import pandas as pd

# Load the dataset
df = pd.read_csv('merged_data20s (1).csv')

# Display the first few rows and info
print(df.head())
print(df.info())

# Check unique values in 'ioc' to confirm it's country codes
print(df['ioc'].unique())

   ranking_date  rank  points  player_id name_first name_last hand  ioc  \
0      20200106     1    9985     104745     Rafael     Nadal    L  ESP   
1      20200106     2    9055     104925      Novak  Djokovic    R  SRB   
2      20200106     3    6590     103819      Roger   Federer    R  SUI   
3      20200106     4    5825     106233    Dominic     Thiem    R  AUT   
4      20200106     5    5705     106421     Daniil  Medvedev    R  RUS   

   height  
0   185.0  
1   188.0  
2   185.0  
3   185.0  
4   198.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332942 entries, 0 to 332941
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ranking_date  332942 non-null  int64  
 1   rank          332942 non-null  int64  
 2   points        332942 non-null  int64  
 3   player_id     332942 non-null  int64  
 4   name_first    332942 non-null  object 
 5   name_last     332942 non-null  object 
 6   hand      

In [35]:
# Check date range
print(df['ranking_date'].min(), df['ranking_date'].max())

# Filter for the most recent ranking date
latest_date = df['ranking_date'].max()
df_latest = df[df['ranking_date'] == latest_date]

# Idea 1 & 2: Total points and Player count by country
country_stats = df_latest.groupby('ioc').agg(
    total_points=('points', 'sum'),
    player_count=('player_id', 'count'),
    avg_height=('height', 'mean'),
    avg_rank=('rank', 'mean')
).reset_index()

# Calculate Top 100 players count
top_100_stats = df_latest[df_latest['rank'] <= 100].groupby('ioc').size().reset_index(name='top_100_count')

# Merge top 100 count
country_stats = country_stats.merge(top_100_stats, on='ioc', how='left').fillna(0)

# Sort by total points to see top countries
print(country_stats.sort_values(by='total_points', ascending=False).head(10))
print(country_stats.describe())

20200106 20231225
    ioc  total_points  player_count  avg_height     avg_rank  top_100_count
94  USA         26777           206  183.317241  1030.694175           10.0
31  FRA         22635           145  185.219048   858.468966           12.0
45  ITA         22059           167  184.405405  1009.425150            5.0
78  RUS         20095            66  186.666667  1081.909091            6.0
28  ESP         18865            91  182.224138   962.769231            7.0
82  SRB         15479            29  185.615385  1032.965517            4.0
2   ARG         14755            87  181.123288   887.793103            6.0
3   AUS         13906            77  183.564516   916.155844            8.0
34  GER         12143            94  187.152542  1023.500000            6.0
32  GBR          9114            77  185.145455   985.974026            4.0
       total_points  player_count  avg_height     avg_rank  top_100_count
count     99.000000     99.000000   99.000000    99.000000      99.00000

In [36]:
# Filter for the latest date again to be sure
latest_date = df['ranking_date'].max()
df_latest = df[df['ranking_date'] == latest_date].copy()

# Clean height: replace 0 with NaN
df_latest['height'] = df_latest['height'].replace(0, pd.NA)

# 1. Basic Aggregations
agg_stats = df_latest.groupby('ioc').agg(
    total_points=('points', 'sum'),
    player_count=('player_id', 'count'),
    average_rank=('rank', 'mean'),
    average_height=('height', 'mean')
).reset_index()

# 2. Top 100 Count
top_100 = df_latest[df_latest['rank'] <= 100].groupby('ioc').size().reset_index(name='top_100_players')

# 3. Best Player Info
# Sort by rank (ascending) to get the best player first
df_sorted = df_latest.sort_values('rank')
best_players = df_sorted.groupby('ioc').first()[['name_first', 'name_last', 'rank']].reset_index()
best_players['best_player_name'] = best_players['name_first'] + ' ' + best_players['name_last']
best_players = best_players.rename(columns={'rank': 'best_player_rank'})

# Merge everything
final_df = agg_stats.merge(top_100, on='ioc', how='left')
final_df = final_df.merge(best_players[['ioc', 'best_player_name', 'best_player_rank']], on='ioc', how='left')

# Fill NaNs for Top 100 count with 0
final_df['top_100_players'] = final_df['top_100_players'].fillna(0)

# Round numerical columns for display
final_df['average_height'] = final_df['average_height'].round(1)
final_df['average_rank'] = final_df['average_rank'].round(0)

# Sort by Total Points for preview
final_df = final_df.sort_values(by='total_points', ascending=False)

# Save to CSV
final_df.to_csv('tableau_choropleth_data20s.csv', index=False)

print(final_df.head())

    ioc  total_points  player_count  average_rank  average_height  \
94  USA         26777           206        1031.0           183.3   
31  FRA         22635           145         858.0           185.2   
45  ITA         22059           167        1009.0           184.4   
78  RUS         20095            66        1082.0           186.7   
28  ESP         18865            91         963.0           182.2   

    top_100_players best_player_name  best_player_rank  
94             10.0     Taylor Fritz                10  
31             12.0      Ugo Humbert                20  
45              5.0    Jannik Sinner                 4  
78              6.0  Daniil Medvedev                 3  
28              7.0   Carlos Alcaraz                 2  


In [37]:
import pandas as pd

# Load the dataset
df = pd.read_csv('merged_data10s.csv')

# Display the first few rows and info
print(df.head())
print(df.info())

# Check unique values in 'ioc' to confirm it's country codes
print(df['ioc'].unique())

   ranking_date  rank   points  player_id   name_first  name_last hand  ioc  \
0      20100104     1  10550.0     103819        Roger    Federer    R  SUI   
1      20100104     2   9205.0     104745       Rafael      Nadal    L  ESP   
2      20100104     3   8310.0     104925        Novak   Djokovic    R  SRB   
3      20100104     4   7030.0     104918         Andy     Murray    R  GBR   
4      20100104     5   6785.0     105223  Juan Martin  del Potro    R  ARG   

   height  
0   185.0  
1   185.0  
2   188.0  
3   191.0  
4   198.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915618 entries, 0 to 915617
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ranking_date  915618 non-null  int64  
 1   rank          915618 non-null  int64  
 2   points        915385 non-null  float64
 3   player_id     915618 non-null  int64  
 4   name_first    915618 non-null  object 
 5   name_last     915618 non-null 

In [38]:
# Check date range
print(df['ranking_date'].min(), df['ranking_date'].max())

# Filter for the most recent ranking date
latest_date = df['ranking_date'].max()
df_latest = df[df['ranking_date'] == latest_date]

# Idea 1 & 2: Total points and Player count by country
country_stats = df_latest.groupby('ioc').agg(
    total_points=('points', 'sum'),
    player_count=('player_id', 'count'),
    avg_height=('height', 'mean'),
    avg_rank=('rank', 'mean')
).reset_index()

# Calculate Top 100 players count
top_100_stats = df_latest[df_latest['rank'] <= 100].groupby('ioc').size().reset_index(name='top_100_count')

# Merge top 100 count
country_stats = country_stats.merge(top_100_stats, on='ioc', how='left').fillna(0)

# Sort by total points to see top countries
print(country_stats.sort_values(by='total_points', ascending=False).head(10))
print(country_stats.describe())

20100104 20191230
    ioc  total_points  player_count  avg_height     avg_rank  top_100_count
31  ESP       24405.0            96  182.508197   869.000000           10.0
34  FRA       19907.0           136  185.425532   858.955882           12.0
96  USA       16394.0           175  185.264463   951.548571            8.0
48  ITA       16074.0           139  184.083333   934.553957            8.0
84  SRB       15316.0            30  185.250000   854.500000            5.0
80  RUS       12030.0            66  185.976190   910.590909            3.0
2   ARG       11099.0            88  181.866667   888.750000            5.0
37  GER       11073.0            78  187.941176   876.230769            4.0
3   AUS       10383.0            59  185.382979   783.728814            6.0
85  SUI        9802.0            31  183.761905  1009.548387            2.0
       total_points  player_count  avg_height     avg_rank  top_100_count
count    101.000000    101.000000  101.000000   101.000000     101.00000

In [39]:
# Filter for the latest date again to be sure
latest_date = df['ranking_date'].max()
df_latest = df[df['ranking_date'] == latest_date].copy()

# Clean height: replace 0 with NaN
df_latest['height'] = df_latest['height'].replace(0, pd.NA)

# 1. Basic Aggregations
agg_stats = df_latest.groupby('ioc').agg(
    total_points=('points', 'sum'),
    player_count=('player_id', 'count'),
    average_rank=('rank', 'mean'),
    average_height=('height', 'mean')
).reset_index()

# 2. Top 100 Count
top_100 = df_latest[df_latest['rank'] <= 100].groupby('ioc').size().reset_index(name='top_100_players')

# 3. Best Player Info
# Sort by rank (ascending) to get the best player first
df_sorted = df_latest.sort_values('rank')
best_players = df_sorted.groupby('ioc').first()[['name_first', 'name_last', 'rank']].reset_index()
best_players['best_player_name'] = best_players['name_first'] + ' ' + best_players['name_last']
best_players = best_players.rename(columns={'rank': 'best_player_rank'})

# Merge everything
final_df = agg_stats.merge(top_100, on='ioc', how='left')
final_df = final_df.merge(best_players[['ioc', 'best_player_name', 'best_player_rank']], on='ioc', how='left')

# Fill NaNs for Top 100 count with 0
final_df['top_100_players'] = final_df['top_100_players'].fillna(0)

# Round numerical columns for display
final_df['average_height'] = final_df['average_height'].round(1)
final_df['average_rank'] = final_df['average_rank'].round(0)

# Sort by Total Points for preview
final_df = final_df.sort_values(by='total_points', ascending=False)

# Save to CSV
final_df.to_csv('tableau_choropleth_data10s.csv', index=False)

print(final_df.head())

    ioc  total_points  player_count  average_rank  average_height  \
31  ESP       24405.0            96         869.0           182.5   
34  FRA       19907.0           136         859.0           185.4   
96  USA       16394.0           175         952.0           185.3   
48  ITA       16074.0           139         935.0           184.1   
84  SRB       15316.0            30         854.0           185.2   

    top_100_players   best_player_name  best_player_rank  
31             10.0       Rafael Nadal                 1  
34             12.0       Gael Monfils                10  
96              8.0         John Isner                19  
48              8.0  Matteo Berrettini                 8  
84              5.0     Novak Djokovic                 2  


In [40]:
import pandas as pd

# Load the dataset
df = pd.read_csv('merged_data00s (1).csv')

# Display the first few rows and info
print(df.head())
print(df.info())

# Check unique values in 'ioc' to confirm it's country codes
print(df['ioc'].unique())

   ranking_date  rank  points  player_id name_first   name_last hand  ioc  \
0      20000110     1  4135.0     101736      Andre      Agassi    R  USA   
1      20000110     2  2915.0     102338    Yevgeny  Kafelnikov    R  RUS   
2      20000110     3  2419.0     101948       Pete     Sampras    R  USA   
3      20000110     4  2184.0     103017    Nicolas      Kiefer    R  GER   
4      20000110     5  2169.0     102856    Gustavo     Kuerten    R  BRA   

   height  
0   180.0  
1   190.0  
2   185.0  
3   183.0  
4   190.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920907 entries, 0 to 920906
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ranking_date  920907 non-null  int64  
 1   rank          920907 non-null  int64  
 2   points        920391 non-null  float64
 3   player_id     920907 non-null  int64  
 4   name_first    920907 non-null  object 
 5   name_last     920907 non-null  object 
 6 

In [41]:
# Check date range
print(df['ranking_date'].min(), df['ranking_date'].max())

# Filter for the most recent ranking date
latest_date = df['ranking_date'].max()
df_latest = df[df['ranking_date'] == latest_date]

# Idea 1 & 2: Total points and Player count by country
country_stats = df_latest.groupby('ioc').agg(
    total_points=('points', 'sum'),
    player_count=('player_id', 'count'),
    avg_height=('height', 'mean'),
    avg_rank=('rank', 'mean')
).reset_index()

# Calculate Top 100 players count
top_100_stats = df_latest[df_latest['rank'] <= 100].groupby('ioc').size().reset_index(name='top_100_count')

# Merge top 100 count
country_stats = country_stats.merge(top_100_stats, on='ioc', how='left').fillna(0)

# Sort by total points to see top countries
print(country_stats.sort_values(by='total_points', ascending=False).head(10))
print(country_stats.describe())

20000110 20091228
    ioc  total_points  player_count  avg_height    avg_rank  top_100_count
31  ESP       30715.0            92  181.850000  715.934783           12.0
34  FRA       21628.0           105  184.673077  742.609524           12.0
3   ARG       18455.0           105  182.100000  779.733333            9.0
96  USA       16379.0           106  185.415094  851.273585            9.0
37  GER       13956.0            92  185.268293  834.456522           11.0
84  SUI       13834.0            21  183.636364  827.476190            3.0
83  SRB       11524.0            26  185.666667  737.653846            3.0
78  RUS       11148.0            61  183.809524  936.524590            4.0
35  GBR        8787.0            52  185.750000  986.807692            1.0
25  CZE        7619.0            46  186.166667  824.586957            2.0
       total_points  player_count  avg_height     avg_rank  top_100_count
count    101.000000    101.000000  101.000000   101.000000     101.000000
mean    2

In [42]:
# Filter for the latest date again to be sure
latest_date = df['ranking_date'].max()
df_latest = df[df['ranking_date'] == latest_date].copy()

# Clean height: replace 0 with NaN
df_latest['height'] = df_latest['height'].replace(0, pd.NA)

# 1. Basic Aggregations
agg_stats = df_latest.groupby('ioc').agg(
    total_points=('points', 'sum'),
    player_count=('player_id', 'count'),
    average_rank=('rank', 'mean'),
    average_height=('height', 'mean')
).reset_index()

# 2. Top 100 Count
top_100 = df_latest[df_latest['rank'] <= 100].groupby('ioc').size().reset_index(name='top_100_players')

# 3. Best Player Info
# Sort by rank (ascending) to get the best player first
df_sorted = df_latest.sort_values('rank')
best_players = df_sorted.groupby('ioc').first()[['name_first', 'name_last', 'rank']].reset_index()
best_players['best_player_name'] = best_players['name_first'] + ' ' + best_players['name_last']
best_players = best_players.rename(columns={'rank': 'best_player_rank'})

# Merge everything
final_df = agg_stats.merge(top_100, on='ioc', how='left')
final_df = final_df.merge(best_players[['ioc', 'best_player_name', 'best_player_rank']], on='ioc', how='left')

# Fill NaNs for Top 100 count with 0
final_df['top_100_players'] = final_df['top_100_players'].fillna(0)

# Round numerical columns for display
final_df['average_height'] = final_df['average_height'].round(1)
final_df['average_rank'] = final_df['average_rank'].round(0)

# Sort by Total Points for preview
final_df = final_df.sort_values(by='total_points', ascending=False)

# Save to CSV
final_df.to_csv('tableau_choropleth_data00s.csv', index=False)

print(final_df.head())

    ioc  total_points  player_count  average_rank  average_height  \
31  ESP       30715.0            92         716.0           181.8   
34  FRA       21628.0           105         743.0           184.7   
3   ARG       18455.0           105         780.0           182.1   
96  USA       16379.0           106         851.0           185.4   
37  GER       13956.0            92         834.0           185.3   

    top_100_players       best_player_name  best_player_rank  
31             12.0           Rafael Nadal                 2  
34             12.0     Jo-Wilfried Tsonga                10  
3               9.0  Juan Martin del Potro                 5  
96              9.0           Andy Roddick                 7  
37             11.0             Tommy Haas                18  


In [43]:
import pandas as pd

# Load the dataset
df = pd.read_csv('merged_data90s.csv')

# Display the first few rows and info
print(df.head())
print(df.info())

# Check unique values in 'ioc' to confirm it's country codes
print(df['ioc'].unique())

   ranking_date  rank  points  player_id name_first name_last hand  ioc  \
0      19900101     1  2913.0   100656.0       Ivan     Lendl    R  USA   
1      19900101     2  2279.0   101414.0      Boris    Becker    R  GER   
2      19900101     3  2111.0   101222.0     Stefan    Edberg    R  SWE   
3      19900101     4  1398.0   100763.0       Brad   Gilbert    R  USA   
4      19900101     5  1354.0   100581.0       John   McEnroe    L  USA   

   height  
0   188.0  
1   190.0  
2   188.0  
3   185.0  
4   180.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725606 entries, 0 to 725605
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ranking_date  725606 non-null  int64  
 1   rank          725606 non-null  int64  
 2   points        655810 non-null  float64
 3   player_id     725578 non-null  float64
 4   name_first    725578 non-null  object 
 5   name_last     725578 non-null  object 
 6   hand      

In [44]:
# Check date range
print(df['ranking_date'].min(), df['ranking_date'].max())

# Filter for the most recent ranking date
latest_date = df['ranking_date'].max()
df_latest = df[df['ranking_date'] == latest_date]

# Idea 1 & 2: Total points and Player count by country
country_stats = df_latest.groupby('ioc').agg(
    total_points=('points', 'sum'),
    player_count=('player_id', 'count'),
    avg_height=('height', 'mean'),
    avg_rank=('rank', 'mean')
).reset_index()

# Calculate Top 100 players count
top_100_stats = df_latest[df_latest['rank'] <= 100].groupby('ioc').size().reset_index(name='top_100_count')

# Merge top 100 count
country_stats = country_stats.merge(top_100_stats, on='ioc', how='left').fillna(0)

# Sort by total points to see top countries
print(country_stats.sort_values(by='total_points', ascending=False).head(10))
print(country_stats.describe())

19900101 19991227
    ioc  total_points  player_count  avg_height    avg_rank  top_100_count
82  USA       22852.0           135  184.412698  792.244444           12.0
27  ESP       18301.0           112  180.629630  648.517857           12.0
30  FRA       13989.0            96  183.222222  678.354167            8.0
33  GER       12631.0           116  184.955556  818.681034            5.0
4   AUS       10447.0            77  183.742857  703.428571            6.0
74  SWE       10204.0            46  185.428571  633.304348            8.0
2   ARG        9827.0            76  181.375000  600.750000            5.0
23  CZE        6949.0            48  186.269231  580.916667            3.0
43  ITA        6372.0            88  182.388889  770.545455            3.0
68  RUS        6291.0            27  183.769231  707.074074            2.0
       total_points  player_count  avg_height     avg_rank  top_100_count
count      87.00000     87.000000   87.000000    87.000000      87.000000
mean     

In [45]:
# Filter for the latest date again to be sure
latest_date = df['ranking_date'].max()
df_latest = df[df['ranking_date'] == latest_date].copy()

# Clean height: replace 0 with NaN
df_latest['height'] = df_latest['height'].replace(0, pd.NA)

# 1. Basic Aggregations
agg_stats = df_latest.groupby('ioc').agg(
    total_points=('points', 'sum'),
    player_count=('player_id', 'count'),
    average_rank=('rank', 'mean'),
    average_height=('height', 'mean')
).reset_index()

# 2. Top 100 Count
top_100 = df_latest[df_latest['rank'] <= 100].groupby('ioc').size().reset_index(name='top_100_players')

# 3. Best Player Info
# Sort by rank (ascending) to get the best player first
df_sorted = df_latest.sort_values('rank')
best_players = df_sorted.groupby('ioc').first()[['name_first', 'name_last', 'rank']].reset_index()
best_players['best_player_name'] = best_players['name_first'] + ' ' + best_players['name_last']
best_players = best_players.rename(columns={'rank': 'best_player_rank'})

# Merge everything
final_df = agg_stats.merge(top_100, on='ioc', how='left')
final_df = final_df.merge(best_players[['ioc', 'best_player_name', 'best_player_rank']], on='ioc', how='left')

# Fill NaNs for Top 100 count with 0
final_df['top_100_players'] = final_df['top_100_players'].fillna(0)

# Round numerical columns for display
final_df['average_height'] = final_df['average_height'].round(1)
final_df['average_rank'] = final_df['average_rank'].round(0)

# Sort by Total Points for preview
final_df = final_df.sort_values(by='total_points', ascending=False)

# Save to CSV
final_df.to_csv('tableau_choropleth_data90s.csv', index=False)

print(final_df.head())

    ioc  total_points  player_count  average_rank  average_height  \
82  USA       22852.0           135         792.0           184.4   
27  ESP       18301.0           112         649.0           180.6   
30  FRA       13989.0            96         678.0           183.2   
33  GER       12631.0           116         819.0           185.0   
4   AUS       10447.0            77         703.0           183.7   

    top_100_players best_player_name  best_player_rank  
82             12.0     Andre Agassi                 1  
27             12.0     Albert Costa                18  
30              8.0   Cedric Pioline                13  
33              5.0   Nicolas Kiefer                 6  
4               6.0   Patrick Rafter                16  


In [46]:
import pandas as pd

# Load the dataset
df = pd.read_csv('merged_data80s.csv')

# Display the first few rows and info
print(df.head())
print(df.info())

# Check unique values in 'ioc' to confirm it's country codes
print(df['ioc'].unique())

   ranking_date  rank  points  player_id name_first name_last hand  ioc  \
0      19801222     1     NaN   100437.0      Bjorn      Borg    R  SWE   
1      19801222     2     NaN   100581.0       John   McEnroe    L  USA   
2      19801222     3     NaN   100284.0      Jimmy   Connors    L  USA   
3      19801222     4     NaN   100282.0  Guillermo     Vilas    L  ARG   
4      19801222     5     NaN   100428.0       Gene     Mayer    R  USA   

   height  
0   180.0  
1   180.0  
2   178.0  
3   180.0  
4   183.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284809 entries, 0 to 284808
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ranking_date  284809 non-null  int64  
 1   rank          284809 non-null  int64  
 2   points        0 non-null       float64
 3   player_id     284793 non-null  float64
 4   name_first    284793 non-null  object 
 5   name_last     284793 non-null  object 
 6   hand      

In [47]:
# Check date range
print(df['ranking_date'].min(), df['ranking_date'].max())

# Filter for the most recent ranking date
latest_date = df['ranking_date'].max()
df_latest = df[df['ranking_date'] == latest_date]

# Idea 1 & 2: Total points and Player count by country
country_stats = df_latest.groupby('ioc').agg(
    total_points=('points', 'sum'),
    player_count=('player_id', 'count'),
    avg_height=('height', 'mean'),
    avg_rank=('rank', 'mean')
).reset_index()

# Calculate Top 100 players count
top_100_stats = df_latest[df_latest['rank'] <= 100].groupby('ioc').size().reset_index(name='top_100_count')

# Merge top 100 count
country_stats = country_stats.merge(top_100_stats, on='ioc', how='left').fillna(0)

# Sort by total points to see top countries
print(country_stats.sort_values(by='total_points', ascending=False).head(10))
print(country_stats.describe())

19800107 19891218
   ioc  total_points  player_count  avg_height     avg_rank  top_100_count
0  ALG           0.0             1    0.000000  1103.000000            0.0
1  ARG           0.0            44  181.551724   537.250000            5.0
2  AUS           0.0            65  183.568182   617.000000            4.0
3  AUT           0.0            25  181.727273   619.120000            3.0
4  BAH           0.0             2  187.500000   308.500000            0.0
5  BEL           0.0            12  187.600000   691.583333            0.0
6  BRA           0.0            54  183.269231   676.592593            1.0
7  BUL           0.0             6    0.000000   866.333333            0.0
8  CAN           0.0            20  184.153846   701.350000            2.0
9  CHI           0.0            18  174.000000   662.277778            0.0
       total_points  player_count  avg_height     avg_rank  top_100_count
count          75.0     75.000000   75.000000    75.000000      75.000000
mean     

In [48]:
# Filter for the latest date again to be sure
latest_date = df['ranking_date'].max()
df_latest = df[df['ranking_date'] == latest_date].copy()

# Clean height: replace 0 with NaN
df_latest['height'] = df_latest['height'].replace(0, pd.NA)

# 1. Basic Aggregations
agg_stats = df_latest.groupby('ioc').agg(
    total_points=('points', 'sum'),
    player_count=('player_id', 'count'),
    average_rank=('rank', 'mean'),
    average_height=('height', 'mean')
).reset_index()

# 2. Top 100 Count
top_100 = df_latest[df_latest['rank'] <= 100].groupby('ioc').size().reset_index(name='top_100_players')

# 3. Best Player Info
# Sort by rank (ascending) to get the best player first
df_sorted = df_latest.sort_values('rank')
best_players = df_sorted.groupby('ioc').first()[['name_first', 'name_last', 'rank']].reset_index()
best_players['best_player_name'] = best_players['name_first'] + ' ' + best_players['name_last']
best_players = best_players.rename(columns={'rank': 'best_player_rank'})

# Merge everything
final_df = agg_stats.merge(top_100, on='ioc', how='left')
final_df = final_df.merge(best_players[['ioc', 'best_player_name', 'best_player_rank']], on='ioc', how='left')

# Fill NaNs for Top 100 count with 0
final_df['top_100_players'] = final_df['top_100_players'].fillna(0)

# Round numerical columns for display
final_df['average_height'] = final_df['average_height'].round(1)
final_df['average_rank'] = final_df['average_rank'].round(0)

# Sort by Total Points for preview
final_df = final_df.sort_values(by='total_points', ascending=False)

# Save to CSV
final_df.to_csv('tableau_choropleth_data80s.csv', index=False)

print(final_df.head())

   ioc  total_points  player_count  average_rank  average_height  \
0  ALG           0.0             1        1103.0             NaN   
1  ARG           0.0            44         537.0           181.6   
2  AUS           0.0            65         617.0           183.6   
3  AUT           0.0            25         619.0           181.7   
4  BAH           0.0             2         308.0           187.5   

   top_100_players best_player_name  best_player_rank  
0              0.0     Moncef Zehar              1103  
1              5.0  Alberto Mancini                 9  
2              4.0      Wally Masur                42  
3              3.0    Thomas Muster                21  
4              0.0      Roger Smith               187  


In [49]:
import pandas as pd

# Load the dataset
df = pd.read_csv('merged_data70s (1).csv')

# Display the first few rows and info
print(df.head())
print(df.info())

# Check unique values in 'ioc' to confirm it's country codes
print(df['ioc'].unique())

   ranking_date  rank  points  player_id name_first       name_last hand  ioc  \
0      19730827   129     NaN     100005    Richard        Gonzalez    R  USA   
1      19730827   114     NaN     100011     Torben          Ulrich    L  DEN   
2      19730827     6     NaN     100016        Ken        Rosewall    R  AUS   
3      19730827    19     NaN     100022        Roy         Emerson    R  AUS   
4      19730827    82     NaN     100025      Barry  Phillips Moore    R  AUS   

   height  
0   188.0  
1   180.0  
2   175.0  
3   183.0  
4   173.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20726 entries, 0 to 20725
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ranking_date  20726 non-null  int64  
 1   rank          20726 non-null  int64  
 2   points        0 non-null      float64
 3   player_id     20726 non-null  int64  
 4   name_first    20726 non-null  object 
 5   name_last     20726 non-nul

In [50]:
# Check date range
print(df['ranking_date'].min(), df['ranking_date'].max())

# Filter for the most recent ranking date
latest_date = df['ranking_date'].max()
df_latest = df[df['ranking_date'] == latest_date]

# Idea 1 & 2: Total points and Player count by country
country_stats = df_latest.groupby('ioc').agg(
    total_points=('points', 'sum'),
    player_count=('player_id', 'count'),
    avg_height=('height', 'mean'),
    avg_rank=('rank', 'mean')
).reset_index()

# Calculate Top 100 players count
top_100_stats = df_latest[df_latest['rank'] <= 100].groupby('ioc').size().reset_index(name='top_100_count')

# Merge top 100 count
country_stats = country_stats.merge(top_100_stats, on='ioc', how='left').fillna(0)

# Sort by total points to see top countries
print(country_stats.sort_values(by='total_points', ascending=False).head(10))
print(country_stats.describe())

19730827 19791226
   ioc  total_points  player_count  avg_height    avg_rank  top_100_count
0  ARG           0.0            26  177.818182  372.384615            3.0
1  AUS           0.0            83  183.030303  371.349398           11.0
2  AUT           0.0            11  180.600000  377.636364            2.0
3  BEL           0.0             5  185.000000  404.600000            0.0
4  BER           0.0             1  188.000000  546.000000            0.0
5  BOL           0.0             3  170.000000  250.000000            0.0
6  BRA           0.0            30  183.666667  398.700000            2.0
7  BUR           0.0             1    0.000000  579.000000            0.0
8  CAN           0.0            10  179.200000  485.900000            0.0
9  CHI           0.0            13  178.200000  302.692308            2.0
       total_points  player_count  avg_height    avg_rank  top_100_count
count          54.0     54.000000   54.000000   54.000000      54.000000
mean            0.0   

In [51]:
# Filter for the latest date again to be sure
latest_date = df['ranking_date'].max()
df_latest = df[df['ranking_date'] == latest_date].copy()

# Clean height: replace 0 with NaN
df_latest['height'] = df_latest['height'].replace(0, pd.NA)

# 1. Basic Aggregations
agg_stats = df_latest.groupby('ioc').agg(
    total_points=('points', 'sum'),
    player_count=('player_id', 'count'),
    average_rank=('rank', 'mean'),
    average_height=('height', 'mean')
).reset_index()

# 2. Top 100 Count
top_100 = df_latest[df_latest['rank'] <= 100].groupby('ioc').size().reset_index(name='top_100_players')

# 3. Best Player Info
# Sort by rank (ascending) to get the best player first
df_sorted = df_latest.sort_values('rank')
best_players = df_sorted.groupby('ioc').first()[['name_first', 'name_last', 'rank']].reset_index()
best_players['best_player_name'] = best_players['name_first'] + ' ' + best_players['name_last']
best_players = best_players.rename(columns={'rank': 'best_player_rank'})

# Merge everything
final_df = agg_stats.merge(top_100, on='ioc', how='left')
final_df = final_df.merge(best_players[['ioc', 'best_player_name', 'best_player_rank']], on='ioc', how='left')

# Fill NaNs for Top 100 count with 0
final_df['top_100_players'] = final_df['top_100_players'].fillna(0)

# Round numerical columns for display
final_df['average_height'] = final_df['average_height'].round(1)
final_df['average_rank'] = final_df['average_rank'].round(0)

# Sort by Total Points for preview
final_df = final_df.sort_values(by='total_points', ascending=False)

# Save to CSV
final_df.to_csv('tableau_choropleth_data70s.csv', index=False)

print(final_df.head())

   ioc  total_points  player_count  average_rank  average_height  \
0  ARG           0.0            26         372.0           177.8   
1  AUS           0.0            83         371.0           183.0   
2  AUT           0.0            11         378.0           180.6   
3  BEL           0.0             5         405.0           185.0   
4  BER           0.0             1         546.0           188.0   

   top_100_players best_player_name  best_player_rank  
0              3.0  Guillermo Vilas                 6  
1             11.0   John Alexander                20  
2              2.0      Peter Feigl                47  
3              0.0  Bernard Boileau               189  
4              0.0     Steven Alger               546  
